In [1]:
from sklearn import datasets
import pandas as pd
import numpy as np

### loading data
#### we'll be using the iris dataset for classification

In [2]:
data = datasets.load_iris()
type(data)

sklearn.utils.Bunch

In [3]:
#let us have a look at the dataset for the general understanding of type of data we are working with
data

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [4]:
iris = data
X = iris.data
Y = iris.target

In [5]:
X.shape

(150, 4)

In [6]:
iris.DESCR

'Iris Plants Database\n====================\n\nNotes\n-----\nData Set Characteristics:\n    :Number of Instances: 150 (50 in each of three classes)\n    :Number of Attributes: 4 numeric, predictive attributes and the class\n    :Attribute Information:\n        - sepal length in cm\n        - sepal width in cm\n        - petal length in cm\n        - petal width in cm\n        - class:\n                - Iris-Setosa\n                - Iris-Versicolour\n                - Iris-Virginica\n    :Summary Statistics:\n\n    ============== ==== ==== ======= ===== ====================\n                    Min  Max   Mean    SD   Class Correlation\n    ============== ==== ==== ======= ===== ====================\n    sepal length:   4.3  7.9   5.84   0.83    0.7826\n    sepal width:    2.0  4.4   3.05   0.43   -0.4194\n    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)\n    petal width:    0.1  2.5   1.20  0.76     0.9565  (high!)\n    ============== ==== ==== ======= ===== ============

In [7]:
df = pd.DataFrame(X)
print(iris.feature_names)
df.columns = iris.feature_names
df.describe()

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [8]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [9]:
features = iris.feature_names
features

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [10]:
type(features)

list

### functions required for implementation of the tree
##### Note: 
Information Gain func. has been used twice with slight modification to the second one. This is because we are dealing with continuous values in our dataset, hence we need to know at what value of a selected feature should we split the classes. Hence, the first function is to give information gain so that we can decide the split point. And the second function is so that we can decide the feature to split upon.

In [11]:
#function to calculate- Entropy

def entropy(classes,counts):
    #counting total no of classes in the node
    tot = counts.sum()
    p = np.zeros(len(classes))
    for i in range(len(classes)):
        #calculating the probablities of each category in the classes present in the node
        p[i] = float(counts[i])/tot
    
    #calculating entropy by formula
    ent = -1 * p * np.log2(p)
    return ent.sum()

In [12]:
#helper function to calculate- Information Gain (FOR FINDING THE SPLIT POINT)

def split_info_gain(X,Y,sel_feat):
    #calculating the old entropy
    classes, counts = np.unique(Y, return_counts=True)
    old_ent = entropy(classes, counts)
    
    #calculating the new entropy
    f_num = features.index(sel_feat)
    cl, co = np.unique(X[:,f_num], return_counts=True) #calculating the no of classes and their counts in the next split
    tot = co.sum()
    
    new_ent = np.zeros(len(cl))
    wt = np.zeros(len(cl)) #array to store respective weights for each node
    
    for i in range(len(cl)):
        y_ = Y[(X[:,f_num] == cl[i])]
        classes, counts = np.unique(y_, return_counts=True)
        new_ent[i] = entropy(classes, counts)
        #calculating weights of each nodes for weighted addition later
        wt[i] = float(co[i])/tot
        
    #new entropy is the weighted sum of entropies in all nodes in the next level
    new_ent =  wt * new_ent
    new_ent = new_ent.sum()

    return (old_ent - new_ent)

In [13]:
#function to return the right split point for a selected feature

def split_point(X,Y,sel_feat):
    f_num = features.index(sel_feat)
    x_ = np.copy(X[:,f_num])
    
    #finding the mid-points of all data points in the selected feature column 
    mid = np.zeros(len(x_)-1)
    for i in range(len(x_)-2):
        mid[i] = float(x_[i] + x_[i+1])/2
    
    max_info_gain = -1
    sp_pt = -1
    x = np.copy(X)
    y = np.copy(Y)
    
    for m in mid:
        #converting continuous values to categorical by checking if they lie above or below a certain mid-point
        x_[x_ < m] = m-1
        x_[x_ >= m] = m
        x[:,f_num] = x_
        #finding out which mid point in the whole column gives maximum gain in information
        i_g = split_info_gain(x,y,sel_feat)
        if (i_g > max_info_gain):
            max_info_gain = i_g
            sp_pt = m #storing split point
        #reinitializing dummy variables to original values for next iteration
        x_ = np.copy(X[:,f_num])
        x = np.copy(X)
        y = np.copy(Y)
        
    return sp_pt

In [14]:
#function to convert continuous values in the dataset into categorical ones whenever required
def cont_2_cat(X,Y,sel_feat):
    f_num = features.index(sel_feat)
    x = X[:,f_num]
    split_pt = split_point(X,Y,sel_feat)
    #unlike the split_point function above, here we are actually converting contiuous
    #values in the selected column into categorical ones for implementation of the algorithm
    x[x >= split_pt] = split_pt + 1
    x[x < split_pt] = split_pt - 1
    
    return X

In [15]:
#function to calculate- Information Gain (FOR CALCULATING THE GAIN RATIO TO SELECT THE FEATURE TO SPLIT UPON)

def info_gain(X,Y,sel_feat):
    #calculating the old entropy
    classes, counts = np.unique(Y, return_counts=True)
    old_ent = entropy(classes, counts)
    
    #calculating the new entropy
    f_num = features.index(sel_feat)
    #this is the only part that has been changed from the previous helper info_gain function 
    #we make dummy variables and pass them to cont_2_cat as we 
    #don't want the selected feature column to change just yet
    x = np.copy(X)
    #y = np.copy(Y)
    cont_2_cat(x,Y,sel_feat)
    #deviation from the previous helper info_gain function stop here
    #rest everything is same as before
    cl, co = np.unique(x[:,f_num], return_counts=True)
    tot = co.sum()
    
    new_ent = np.zeros(len(cl)) #array to store respective entropies for each node after split
    wt = np.zeros(len(cl)) #array to store respective weights for each node after split
    
    for i in range(len(cl)):
        y_ = Y[(x[:,f_num] == cl[i])]
        classes, counts = np.unique(y_, return_counts=True)
        new_ent[i] = entropy(classes, counts)
        #calculating weights of each nodes for weighted addition later
        wt[i] = float(co[i])/tot
    new_ent =  wt * new_ent
    new_ent = new_ent.sum()
    
    if(new_ent == 0):
        return 0.0 #in case of no split, the entropy remains same, hence info gain is indeed zero
    else:
        return (old_ent - new_ent)

In [16]:
#function to calculate- Split Info

def split_info(X,Y,sel_feat):
    f_num = features.index(sel_feat)
    #we make dummy variables and pass them to cont_2_cat as we 
    #don't want the selected feature column to change just yet
    x = np.copy(X)
    #y = np.copy(Y)
    cont_2_cat(x,Y,sel_feat)
    cl, co = np.unique(x[:,f_num], return_counts=True)
    a = np.zeros(len(cl))
    b = np.zeros(len(cl))
    tot = co.sum()
    #check the split info formula; we need to know all the splits
    #so iterating over all the splits in Y according to the classes in X
    for i in range(len(cl)):
        y = Y[(x[:,f_num] == cl[i])]
        a[i] = float(len(y))/tot
        b[i] = np.log2(a[i])
    
    return (-1 * a * b).sum()

In [17]:
#function to calculate- Gain Ratio    
def gain_ratio(X,Y,sel_feat):
    ig = info_gain(X,Y,sel_feat)
    if(ig > 0): 
        #no info gain means no splitting, hence this condition has been put
        #to stop gain ratio from becoming undefined
        return float(ig)/split_info(X,Y,sel_feat)
    elif(ig <= 0):
        return ig

In [18]:
#function to build decision tree
def dec_tree(X,Y,feat,level):
    #printing the needed information as required 
    print("Level =", level)
    level += 1
    classes, counts = np.unique(Y, return_counts=True)
    for c in range(len(classes)):
        print("Count of ",classes[c]," = ",counts[c])
    print("Current Entropy is =", entropy(classes, counts))
    
    #checking for pure node or if the feature list is exhausted
    if(len(feat) == 0 or len(np.unique(Y)) == 1):
        print("Reached leaf Node")
        print()
        return
    
    
    #initializing the variables necessary for calculations of Gain Ratio, feature to split etc.
    g_r = -1
    max_gain = -1
    sel_feat = ""
    
    #comparing gain ratio for all features and deciding which feature to split upon
    for f in feat:
        g_r = gain_ratio(X, Y, f)
        if(g_r > max_gain):
            max_gain = g_r
            sel_feat = f
    #if pruning is needed, set the flag variable below to 1
    flag = 0
    if(max_gain == 0 and flag == 1):
        print("Reached leaf Node")
        print()
        return
        
    print("Splitting on feature",sel_feat, "with gain ratio", max_gain)
    print()

    f_num = features.index(sel_feat)
    #when feature is finally selected, we actually convert the column into categorical data
    #as now we'll split Y into different nodes based on the selected feature
    cont_2_cat(X,Y,sel_feat)
    
    #feature has been selected, checking the number of categories in the selected feature
    cl = np.unique(X[:,f_num])
    feat.remove(sel_feat)
    #splitting Y into different nodes according to categories found in selected feature
    for i in cl:
        x_tr = X[(X[:,f_num] == i)]
        y_tr = Y[(X[:,f_num] == i)]
        dec_tree(x_tr, y_tr, feat,level)
        
        
    #adding the feature in features list again for previous recursive calls
    feat.insert(f_num,sel_feat)
    

### building the tree

In [19]:
features = iris.feature_names
feat=features.copy()

#building decision tree
dec_tree(X,Y,feat,0)

Level = 0
Count of  0  =  50
Count of  1  =  50
Count of  2  =  50
Current Entropy is = 1.584962500721156
Splitting on feature petal width (cm) with gain ratio 0.9999999999999999

Level = 1
Count of  0  =  50
Current Entropy is = 0.0
Reached leaf Node

Level = 1
Count of  1  =  50
Count of  2  =  50
Current Entropy is = 1.0
Splitting on feature petal length (cm) with gain ratio 0.6621582046482519

Level = 2
Count of  1  =  44
Count of  2  =  1
Current Entropy is = 0.15374218032876188
Splitting on feature sepal length (cm) with gain ratio 0.1886274738620596

Level = 3
Count of  1  =  3
Count of  2  =  1
Current Entropy is = 0.8112781244591328
Splitting on feature sepal width (cm) with gain ratio 0.15106563978903303

Level = 4
Count of  1  =  1
Current Entropy is = 0.0
Reached leaf Node

Level = 4
Count of  1  =  2
Count of  2  =  1
Current Entropy is = 0.9182958340544896
Reached leaf Node

Level = 3
Count of  1  =  41
Current Entropy is = 0.0
Reached leaf Node

Level = 2
Count of  1  = 

<img src="DT.png">